# Module 1 - importing, manipulating, and representing data 
----------------------------------------------------------------------------------------

The basis of any statistical analysis is the underlying data.

A data-set is typically presented as a file containing information formatted as a table:
 * each line correspond to an observation ( individual, sample, ... )
 * each column correspond to a measured variable ( height, sex, gene expression, ... )


To read tabulated data and manipulate them, we will rely on **[pandas](https://pandas.pydata.org/)**, 
a "high-level" module designed for statistics/exploratory data analysis. A great strength of pandas is its **DataFrame** which emulates many of the convenient behavior and syntax of their eponym counterpart in the **R** language.

To graphically represent data, we will rely on **[seaborn](https://seaborn.pydata.org/index.html)**.
Seaborn is designed to work hand-in-hand with pandas DataDrame to produce **efficient data representation** from fairly simple commands. The seaborn official website proposes [very good tutorials](https://seaborn.pydata.org/tutorial.html) as well as a [gallery](https://seaborn.pydata.org/examples/index.html) with associated code to get you started quickly.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("All modules loaded successfully!")

In [ ]:
import matplotlib as mpl
## making the plotted labels a bit bigger for presentation with a projector
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

mpl.rc('font', **font)

# ToC <a id='toc'></a>


1. [Reading the data](#reading)

    1.1. [the basics](#reading.1)

    1.2. [header or not header, that is the question](#reading.2)

    1.3. [setting up the row index](#reading.3)

    1.4. [other options](#reading.4)

    1.5. [more formats](#reading.5)

2. [data manipulation](#manip)

    2.1. [first contact with the data](#manip.1)

    2.2. [accessing specific parts of the data - rows and columns](#manip.2)

    2.3. [accessing specific parts of the data - selection](#manip.3)

    2.4. [Operations on columns](#manip.4)
    
    2.4. [adding/removing and combining columns](#manip.5)

3. [data description and representation](#descr)

    3.1 [basic description - common summary statistics](#descr.1)

    3.2 [Representing one column - histograms and density line](#descr.2)

    3.x.[Interlude : multi-panel figures](#multi)

    3.3 [accounting for categories in the data](#descr.3)

    3.4 [representing the relationship between 2 numerical variables](#descr.4)

4. [writing data and plot](#writing)

5. [Free form exercise](#exo)

6. [tips and tricks](#tricks)


## 1. Reading tabulated data <a id='reading'></a> 

## 1.1 The basics <a id='reading.1'></a> 

What is the file name? location?
What is the saprator between fields?

**`pd.read_table()`** is the generalist pandas function to read tabulated data files. Aside from the name of the file to read, here are some useful optional arguments:
* `sep`: separator between columns (by default `"\t"`)
* `header`: row number(s) to use as the column names. By default the first line is used as header.
  Use `header=None` if the file does not contain column names.
* `skiprows`: Line numbers to skip (0-indexed) or number of lines to skip (int) at the start of the file.
* (true_values/false_values ??)

Of course you can learn (much) more using `help(pd.read_table)`.



Let's try to load the `data/titanic.csv` file. As its name suggest, this table contains data about the ill-fated [Titanic](https://en.wikipedia.org/wiki/Titanic) passengers, travelling from England to New York in April 1912. The data file is named `"titanic.csv"` and like its extension suggests, it contains **C**omma-**S**eparated **V**alues.

Note the usage of the `head()` method of a DataFrame to display only its first 5 lines.

In [ ]:
import pandas as pd

df = pd.read_table("data/titanic.csv") 
df.head()                                 # By default, head() returns the 5 first lines of a DataFrame.

This does not look so great...

### Micro-Exercise
* Try to fix the cell just above by playing with the option(s) of `pd.read_table`.


<br>

<br>

<center>do not scroll or be spoiled!</center>

-----


<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

As you have just seen, `pd.read_table()` expects the input data to be **tab-delimited** (by default). Since this is not the case of the `titanic.csv` file, each line was treated as a single field (column), thus creating a DataFrame with a single column.

As implied by its `.csv` extension (for "comma-separeted values"), the `titanic.csv` file contains **comma-delimited** values.  
To load a CSV file, we can either:
* Specify the separator value in via the **`sep`** argument: `pd.read_table(<file name>, sep=",")`.
* Use **`pd.read_csv()`**, a function that uses commas as default separator.

In [ ]:
df = pd.read_table("data/titanic.csv", sep=",") 
# Alternatively: df = pd.read_csv("data/titanic.csv", sep=",") 
df.head()

<br>
<br>

[back to toc](#toc)

## 1.2 Header or not header, that is the question <a id='reading.2'></a>


Another important aspect of reading data is whether your dataset has a header or not. 
By default, **`pd.read_table()` expects the first line to be a header**, unless you either:
 * Use the argument **`header=None`**.
 * Specify column names using the **`names`** argument.

<br>

Let's try to load a dataset that does not have any header:  
> *Note:* as illustrated in the cell below, `head()` can print any number of lines by passing a
  value to it - here the first 3 lines)

In [ ]:
df = pd.read_table("data/titanic_no_header.csv", sep=",") 
df.head(3)

Notice how the **values in the first line were used as column names**... this is not ideal.

Let's correct this:

In [ ]:
df = pd.read_table("data/titanic_no_header.csv", 
                   sep=",", header=None) 
df.head(n=3) 

Much better! 

Let's go one step further and **assign our own column names** using the `names` argument:

In [ ]:
df = pd.read_table("data/titanic_no_header.csv", sep=",", 
                   names = ["name", "column2", 
                            "age", "column4", 
                            "blip", "bloop", "spam", "eggs"]) 
# As you can see, we can choose our own names, whether they make sense or not.

df.head(3) 

[back to toc](#toc)

<br>
<br>

## 1.3 Setting the row index <a id='reading.3'></a>

Now that we learned to change column names, let's see how to modify **row names**, which are called the **index**.

> *Note:* not all dataset need a custom index. Oftentimes the default index (numbers starting from 0 
  that correspond to line positions) is enough. It can often also be advantageous to have the index
  correspond to row positions.

There are several options available to modify the index that we will illustrate below.

### 1.3.1 Input file contains row names (file has one less column names than data fields)

Depending on how the data file was produced, it does sometimes already contain row names.

Let's see an example of this with the `data/titanic_implicit_index.csv` file. First we will display the "raw" content of our file:

In [ ]:
with open("data/titanic_implicit_index.csv", "r") as f:
    for x in range(5):
        print(f.readline().strip())

As can be seen above, the 1st line of the file contains 8 elements, while all other lines contain 9 elements! This is because the first field of the file contains the row names.

> Note: if you are using a Linux or MacOS machine, you could also use the one-liner  
>  `!head -n 4 data/titanic_implicit_index.csv`  
> (the leading `!` is a special Jupyter Notebook syntax that indicates a call to a shell program)

In [ ]:
df = pd.read_table("data/titanic_implicit_index.csv", sep=",") 
df.head(n=3)

In [ ]:
df.index[:5]  # access the index directly

When pandas detects this configuration (one more field than column names), it automatically uses the first, nameless, column as index.

<br>

### 1.3.2 Using `index_col` to manually specify the index

Pandas can be instructed to use a specific column of an input file as index via the **`index_col`** argument.

As shown below, the index column can be indicated either:
* By position (*reminder:* column indexing is zero-based).
* By name.

In [ ]:
# Index column is specified by position: 0 = 1st column.
df = pd.read_table("data/titanic.csv", sep=",", index_col=0)
df.head(3) 

In [ ]:
# Index column is specified by name.
df = pd.read_table("data/titanic.csv", sep=",", index_col="Name")
df.head(3)

In [ ]:
df.columns

> *Note:* removing the name of the index ("Name" in the example above) can be done
  with: `df.index.name = None`
  
> *Note:* pandas also has a system of multiple, hierarchised indexing. This is, however, a much
  more specialized and advanced feature.

[back to toc](#toc)

<br>
<br>

## 1.4 Other `pd.read_table()` options <a id='reading.4'></a>

`pd.read_table()` has a vast arrays of options. We cannot go though all of them, but here are a few which may be of interest to you:
* `true_values`/`false_values`, each a list. A must if you have columns encoded with "yes"/"no" labels.
* `na_values`: takes a list. Ideal when your NAs are encoded as something unusual (eg, `.`,` `,`-9999`,...).
* `parse_dates`/`infer_datetime_format`/`date_parser`: options to help you handle date parsing, which can
  otherwise be a nightmare - [more on this here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).
* `compression`: your data is in a compressed format (zip, gzip, ...)? Not a problem!


[back to toc](#toc)

<br>
<br>

## 1.5 Reading other tabulated file formats <a id='reading.5'></a>

As you might expect, pandas is not limited to text, csv/tsv-like files.

* `pd.read_excel()`
* `pd.read_json()`
* `pd.read_sql()` 
* ... see [here for an exhaustive list of pandas reader and writer functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).



In [ ]:
genbank_df = pd.read_json("data/genbank.sub.ndjson", lines=True)
genbank_df.head(3)

<br>

### Micro-Exercise:
* Read the file `data/pbmc_data.countMatrix.50.txt.zip` as a DataFrame. Determine which is the separator,
  and decide whether there is a header and/or an index column.

[back to toc](#toc)

<br>
<br>
<br>

# 2. Data manipulation <a id='manip'></a>
---------------------------------

Now that you know (almost) everything there is to know about loading your data files as a `DataFrame`, let's see what we can actually do with these!


## 2.1 First contact with the data <a id='manip.1'></a>

Gathering basic information about a DataFrame is fairly easy, and we will illustrate this with the titanic data:

In [ ]:
df = pd.read_table("data/titanic.csv", sep=",")
df.head(3)

<br>

* **`df.shape`** returns a tuple with the numbers of rows and columns: `(row_count, col_count)`.

In [ ]:
df.shape

In [ ]:
numberRows, numberCols = df.shape
print('row count:', numberRows, '\ncolumn count:', numberCols)

<br>

* Column names can be accessed with the **`df.columns`** attribute, and the index with the
  **`df.index`** attribute:

> *Note:* here the index is number-based, starting with 0 (this is the default pandas indexing).

In [ ]:
print('column names:\n', df.columns)
print('index:\n', df.index)

<br>

* The **`df.columns`** and **`df.index`** attributes can also be used to set new values for column names
  and index labels.

In [ ]:
df.head(3)

In [ ]:
df.columns = [x.upper() for x in df.columns]
df.index = ["passenger_" + str(i) for i in df.index]
df.head(3)

In [ ]:
# Let's reset out changes in index and column names:
df.columns = df.columns.str.capitalize()
df.index = range(0, df.shape[0])
df.head(3)

In [ ]:
df.columns

> *Note:* how did we apply a `str` function to all the column names at once?
  That is a very powerful feature, which we'll discuss later.

<br>

Each column of a DataFrame has an associated **data type**, which controls the operations you may perform on it. Our example DataFrame contains the following data types:
* `object` : catch all for text, intermixed or not with numbers
* `float64` : float
* `int64` : integer

In [ ]:
print("columns types:\n", df.dtypes, sep="")  # list the type of each column.

Other types exist. The main types are :
    
 * object : catch-all type
 * int64 : integers
 * float64 : floats
 * bool : booleans
 * datetime64 : date and time points 
 * category : categorical data with fixed possible values
 

To change the type of a column, the simplest is to use the `.astype` method:

In [ ]:
## float to object 
FareStr = df.Fare.astype(str)
## it can then be manipulated as a string:
FareStr = '$' + FareStr
FareStr

<br>

### Micro-Exercise:

A typical case: the data you were given is badly formatted and you have to cure this a bit. For example, a column contains percentages with the % sign...


In [ ]:
# here I just generate the data manually
PercentColumn = pd.Series( np.random.randint(0,100,100), dtype="str") +'%' 
PercentColumn

Your mission: get this column into something usable as numbers.

----

In the case of the titanic data, the passenger class is not really a number, and may be better represented as a `category` variable

In [ ]:
df.Pclass = df.Pclass.astype("category")
df.Pclass

Then category-specific functions can be accessed with `.cat`

In [ ]:
# for instance, I may want to change the labels a bit here:
df.Pclass= df.Pclass.cat.rename_categories( {1:'I',
                                             2:"II",
                                             3:"III"})
df.Pclass

[datetime64](https://pandas.pydata.org/docs/user_guide/timeseries.html) and [category](https://towardsdatascience.com/staying-sane-while-adopting-pandas-categorical-datatypes-78dbd19dcd8a) are fairly specific, and we refer you to the provided links if you want to learn more about them.


[back to toc](#toc)

<br>

## 2.2 DataFrame subsetting - accessing specific rows and columns <a id='manip.2'></a>

### 2.2.1 Accessing a single column

One can access a column just by using the syntax **`df[<column name>]`**

In [ ]:
df['Sex']

> *Note:* `pandas` only prints first and last 5 rows of the column to avoid clogging your screen, as well as some useful info.

Alternatively, we can also use the syntax **`df.<column name>`**

In [ ]:
df.Sex

<br>

### 2.2.2 Subsetting a DataFrame with the `loc[]` and `iloc[]` indexers

A very common operation to perform on DataFrames is to create a subset by selecting certain rows and/or columns.  
There are 2 methods in pandas to perform a selection on a DataFrame (here `df`):
* **position based:** using `df.iloc[<row selection>, <column selection>]`
* **index/label based:**  using `df.loc[<row selection>, <column selection>]`

![image.png](img/pandas_position_vs_index_selection.png)

To select all rows/or columns, the symbol `:` can be used as row or column selection. It works with both `.loc[]` and `.iloc[]`:
* `df.loc[<row selection>, :]` - select all columns.
* `df.loc[:, <column selection>]` - select all rows.

> *Note:* when selecting on rows only (i.e. select all columns), the `df.loc[<row selection>, ]`
  and `df.loc[<row selection>]` syntaxes are also possible. The `:` is not compulsory in that case).

<br>

**Common pitfall with slicing:** `loc[]` includes the end index, but `iloc[]` does not:

In [ ]:
df.loc[ 0:3 , : ]    # this selects the first 4 rows.

In [ ]:
df.iloc[ 0:3 , : ]   # this selects the first 3 rows.

<br>

**Examples:** let's apply all we have learned about `loc[]` and `iloc[]`: 

In [ ]:
# Select the first 3 rows, and all columns.
df.loc[0:3, : ]

In [ ]:
# Select the first 3 rows, and columns 'town name' and 'Total'.
df.loc[ : , ['Sex' , 'Name' , 'Name' , 'Age'] ]

> *Note:* I am free to select a column several time, in whichever order I wish. This can thus be used to
  re-order columns in a DataFrame.

In [ ]:
df.loc[ :, "Name":"Fare" ]   # select all rows, and columns from column 'Name' to column "Fare".

In [ ]:
df.iloc[0:2, [0,2,3]]         # select the first 2 rows, and columns 0,2 and 3.

In [ ]:
df.iloc[0, [0,2,3]]           # select the first row, and columns 0,2 and 3.

<br>

**Question:** what type of object do we get when selecting a single row/column? You may have noticed that it does not get represented as a list, so what is it?

In [ ]:
# Select a single row:
row_4 = df.iloc[3,]
row_4b = df.loc[3,]
print(type(row_4))
print(type(row_4b))

# Select a single column. Note that when selecting by columns only, ":" must be used to indicate
# that all rows should be selected.
col_age_a = df.loc[:,"Age"]
col_age_b = df.iloc[:,3]
col_age_c = df["Age"]          # When selecting based on columns only, using this syntax is simpler.
print(type(col_age_a))
print(type(col_age_b))


These are pandas **`Series`**. They are the equivalent of `DataFrame`, but **1-dimensional**.
Their elements can be accessed in quite a similar way:

In [ ]:
print( row_4 )
print( "\n---\n" )
print( row_4[0] )   # by position
print( "\n---\n" )
print( row_4.Age )  # by name 

<br>

### Micro-Exercise
* Select all odd rows from the Titanic data frame, as well as the columns "Name", "Age" and "Fare".
* Re-order the columns so that "Age" is first and "Name" is second.

**Hint:** we will see how to perform conditonal selection later, but for now you can use the `range()` function to help you with this task.

<br>
<br>

[back to toc](#toc)

## 2.3 Accessing specific parts of the data - selection <a id='manip.3'></a>

Another, powerful, way of accessing specific part of the data is by defining a mask, which will **filter the data through a particular condition**.


In [ ]:
maskMale = df['Sex'] == 'male'  
maskMale

The mask (a `Series` object), is in effect a list of values that are `True` or `False` depending on whether or not they satisfy the defined condition (here, `Sex` is equal to "male").

<br>

A great method of `Series` containing categorical kind of data (such as `True`/`False` values only) is **`values_counts()`**.


In [ ]:
maskMale.value_counts()

The mask can also be applied to the `DataFrame`.

In [ ]:
df.loc[maskMale, ['Sex','Fare','Survived']]

> Note that only the 577 rows corresponding to male passengers have been selected.

<br>

Masks may be combined to produce more complex selection criteria, using **`&`** (logical and) and **`|`** (logical or).

In [ ]:
# male passenger with a fare > 200
mask = (df[ 'Sex' ] == 'male') & ( df.Fare > 200  ) 
df.loc[ mask , ]

In [ ]:
# Select all people that are either < 25 or a women.
df.loc[(df.Age < 25) | (df.Sex == "female"), :]

> **Reminder: `.iloc[]` does not support boolean results (True/False) for row selection**: it requires row positions. One can hack its way through that by calling the index of a mask, or using the [query method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html)


### Micro-Exercise:
* Select the fare and name of passengers in first class (`Pclass` "I" ) which are less than 18 years old.

<br>
<br>

[back to toc](#toc)

## 2.4 Operations on columns <a id='manip.4'></a>

Pandas DataFrame allow to use arithmetic operators on columns:

In [ ]:
df.Age

In [ ]:
df["Age"] = df["Age"] + 1
df.Age

In [ ]:
df["Age"] -= 1   # same as df["Age"] = df["Age"] - 1
df.Age

This syntax becomes quite powerful as we can now very easily apply opperations accross whole columns.

For instance, consider this data from the 1880 swiss census:
* The `Total` column gives the total number of registered inhabitants in the given town
* The `Male` columns gives the number of men.

In [ ]:
df_census = pd.read_table("data/swiss_census_1880.csv", 
                          sep=",")
df_census.loc[:5, ["town name", "Total", "Male"]]

<br>

To get the fraction of men in each town, we can now simply write:

In [ ]:
df_census.Male / df_census.Total

<br>

Better yet: we can assign our result to a **new column as if we were adding a key to a dictionnary**:

In [ ]:
df_census['Male Fraction'] = df_census.Male / df_census.Total
df_census.loc[:, ["town name", "Total", "Male", "Male Fraction"]].head()

<br>

These row value assignment operations may be combined with a selection operation, allowing e.g. to modify the values of certain rows in a DataFrame based on a certain condition.

This is particularly useful when you want to mark some data as NAs for instance. Let's magine that for some reason the fares of class 3 are not valid. We want to set them to NA:

In [ ]:
df.head()

In [ ]:
# NA is represented using pd.NA
df.loc[ df.Pclass=="III" , 'Fare'] = pd.NA
df.head()

<br>

### Micro-Exercise:
* Children under the age of 10 get a special discount of 50% on their fare.
  Apply this by dividing by 2 the `Fare` of eligible passenger in the `df` DataFrame.



---

As we have just seen, pandas DataFrame allow easy arithmetics on the elements of a column.

But it is also possible to apply any custom function on the elements of a column using the `.map()` method. E.g.: `df["Survived"].map(func)`

<br>

**Example**

In [ ]:
def sillyFunction( x ):
    ''' returns "even" if x is divisible by 2, otherwise returns "odd" '''
    if x % 2 == 0 :
        return 'even'
    return 'odd'

print( 2 , '->' , sillyFunction(2) )
print( 3 , '->' , sillyFunction(3) )

In [ ]:
## for each value in the age column, apply the sillyFunction
## NB : THIS DOES NOT MODIFY THE CONTENT OF THE DATAFRAME
df.Age.map( sillyFunction )

<br>

### interlude: copy or not copy?

What happens if we select part of a DataFrame and modify it? does the original data stays the same?

In [ ]:
df = pd.read_table("data/titanic.csv", sep=",")

In [ ]:
df.loc[df.Sex == "male", : ].head()

In [ ]:
df.loc[df.Sex == "male", "Age" ] = 999 

In [ ]:
df.head()

Well OK, the age was changed from the slice to the main `DataFrame` : `pandas` avoid doing copies when it can... BUT :

In [ ]:
df_maleOnly = df.loc[ df.Sex == 'male' , : ]

In [ ]:
# Setting the age to 888 in the dataframe of male only
df_maleOnly.Age = 888

What's this? We get a warning!

In [ ]:
df_maleOnly.head()

In [ ]:
df.head()

Indeed the change made to `df_maleOlny` has not been reflected to `df`.


Sadly, it is not always that easy to get when you get a **view** or a **copy**.

![image.png](img/view_copy.png)


 * **view:** this still point to the original data.
 * **copy:** new data. Modifying this leaves the original data untouched.

In general, **using `.loc[]` should return a view**, however that also depends on the evaluation order of some of the performed operations:

In [ ]:
# Does this line create a copy or a view? Nobody knows.
df_maleOnly = df.loc[ df.Sex == "male", : ] 


# Only when we try to set some data does pandas detect something potentially fishy and warns us...
df_maleOnly.Age=888

To get a copy, we must make it explicit to pandas by using the **`copy()`** method:

In [ ]:
# Creates a copy of the DataFrame returned by .loc[]
df_maleOnly = df.loc[ df.Sex == 'male' , : ].copy()


# Now we don't get a warning anymore:
df_maleOnly.Age=888

This issue is quite complex, but as you are likley to encounter this warning at some point, it is better to have the cat out of the bag now. Here is a recommended reading if you wish to get a more [in-depth explaination of view vs. copy](https://www.dataquest.io/blog/settingwithcopywarning).


[back to toc](#toc)

<br>
<br>

## 2.5 Adding/removing and combining columns <a id='manip.5'></a>


In [ ]:
df = pd.read_table( "data/titanic.csv", sep=",")

We have already seen how to create a new column: associate a column name (key) and a container (value) with a number of elements corresponding to the number of rows in the `DataFrame`:

In [ ]:
import numpy.random as rd

df['discount'] = rd.random(df.shape[0])
df.head()

In [ ]:
df.columns

You can also create a column from existing ones:

In [ ]:
df['Discounted_Fare'] = ( 1 - df.discount ) * df.Fare
df.head()

Removing columns is about as easy:

In [ ]:
df.drop(columns='discount' , inplace=True)  # use the 'index' argument to remove rows instead
print("is 'discount' part of the columns : " , 'discount' in df.columns)

Note how we gather the value returned by `df.drop` to get the data frame without the column.
This can be circumvented with `inplace=True`.

In [ ]:
col = 'Discounted_Fare'
df2 = df.drop(columns=col) 

print("is '",col,"' in df : " , col in df.columns , sep='')
print("is '",col,"' in df2 : " , col in df2.columns , sep='')
## default behavior : df is unchanged, df2 is the oone without the column

print('---')
df2 = df.drop(columns=col, inplace = True) 

print("is '",col,"' in df : " , col in df.columns , sep='')
print("df2 is ",df2)
## inplace = True : df is changed, df2 is None

In [ ]:
help( df.drop )


<br>

If you want to **add rows**, then you may use **`pd.concat()`**, which takes a list of `DataFrame` or `Serie` :


In [ ]:
df2 = pd.concat( [df , pd.DataFrame({'Name':'The Mr. Doctor',
                 'Sex':'fluid',
                 'Age':pd.NA,
                 'Pclass':2,
                 'Survived':pd.NA,
                 'Family':pd.NA,
                 'Fare':42.00,
                 'Embarked':pd.NA}, index=[df.shape[0]]) ] )
df2.tail()

In [ ]:
df2.drop(index=891, inplace=True)
df2.tail()

<br>

## Exercise 1 - data manipulation

Using the Titanic dataset:

1. Select passengers which survived. How many are males/females?
2. Create a new column `Title` is the `DataFrame` representing the title by which passengers should
   be addressed. The title can be found in the passenger name and is the only word ending with a `'.'`
   
   **hint for question 2:** there is no *easy, one-line* answer. Create a function to get the title from the name and work your way from there.
   

In [ ]:
m_survived = df.Survived == 1

df.loc[m_survived , 'Sex'].value_counts()


Uncomment the following to load solutions:

In [ ]:
# %load -r 1-3 solutions/solution_01_01.py
# 1. Select passengers which survived. How many are males/females?

In [ ]:
# %load -r 4- solutions/solution_01_01.py
#2. Create a new column Title is the DataFrame representing the title by which passengers should be addressed. The title can be found in the passenger name and is the only word ending with a '.'

[back to toc](#toc)

<br>
<br>
<br>

# 3. Data description and representation <a id='descr'></a>
-----------------------------------------------------------

## 3.1 Basic description - common summary statistics <a id='descr.1'></a>

When doing exploratory analysis of a dataset, it is often useful get some basic statistics on a per-column basis (since rows will typically represent the samples and columns the explanatory variables).

Pandas has a number of methods that can be applied both on an entire DataFrame or on individual columns (Series):
* `describe()`: print summary statistics for all **numeric columns** (count, mean, min, max, std, quantiles).
* `count()`: returns the count of values (in each column, if applied to a DataFrame), that are not NaN
  (i.e. missing values - Not a Number).
* `value_counts()`: returns the count of each value. Useful for discrete variables (e.g. factors).
* `min()`, `max()`, `mean()`, `std()`, `quantile()`, `sum()`: returns the min, max, mean, etc.

When applied to a DataFrame, these method compute values for all columns, skipping columns for which they cannot be applied, e.g. standard deviation can only be computed for numeric columns).

> Note: by default, all these statistics are computed on rows (e.g. max value of all rows in a given column), but they can also be computed by row by adding the `axis=1` argument. E.g. `df.max(axis=1)` returns the max value among columns for each row.

<br>

**Example:** count the number of people that embarked at different ports.
> *Note:* embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
df.Embarked.value_counts()

Another nice trick to know about, you can create contigency tables to look at the relationship between categorical variables:

In [ ]:
pd.crosstab(df.Pclass , df.Survived)

<br>

**Example:** compute the mean, median, min and max fare.

In [ ]:
print('mean Fare  ',df.Fare.mean())
print('median Fare',df.Fare.median())
print('min Fare   ',df.Fare.min())
print('max Fare   ',df.Fare.max())

<br>

Of course we can combine this with selections:

In [ ]:
mask_male = df.Sex == 'male'

print('median fare of male', df.Fare[mask_male].median() )
print('median fare of female', df.Fare[~mask_male].median() )
# note the use of ~ to reverse the mask!

<br>

Another neat trick is that these methods can be used on whole DataFrames. By default they apply on each columns.

In [ ]:
df.mean()   # use df.mean(axis=1) to compute a mean per row, here non-sensical

We get a "FutureWarning", letting us know that in the future this will change. So if we correct:

In [ ]:
df[["Age","Pclass","Family","Fare"]].mean()

<br>

### Micro-Exercise:
* Compute the mean fare for each passenger class (`Pclass`).

<br>

### Micro-Exercise:
* Using the dataset of single cell data `data/pbmc_data.countMatrix.50.txt.zip`:
  1. Compute the sum for each column.
  2. Normalize each column by dividing its values by the column-wise sum.
  


In [ ]:
df_sc = pd.read_table('data/pbmc_data.countMatrix.50.txt.zip', sep=" ", index_col=0)


<br>

A very useful method of DataFrame to get an **overview of a dataset** is **`df.describe()`**:

In [ ]:
df.describe()

The `df.describe()` function gives information about all numerical columns in the dataset at once (note that non-numerical values are absent here).

It is very useful not only to get a first impression on the dataset, but also to catch eventual errors in the data: a negative number where there should be only positive values, missing values (NAs), ...


<br>

**BUT** the most eagle-eyed among you will have spotted a problem here... what is it? how to solve it ?



In [ ]:
# %load solutions/solution_describe.py


<br>

Anyway, back to the matter at hand.

**`.describe()`** gives access to some of the most commonly used summary statistics:
* (arithmetic) **mean** : ${\displaystyle \bar{x}={\frac {1}{n}}\sum _{i=1}^{n}x_{i}}$  or, for coders : `sum(x)/len(x)`
* **standard deviation** (a.k.a. std, stdev) : this corresponds to the average of the absolute difference to the mean. It is the **square root of the variance**.
* **minimum** and **maximum**: smallest and biggest value among the data. Looking at them can help detect outliers.
* **quartiles** : they correspond to the value such that
    * 25% (first quartile, Q1), 
    * 50% (second quartile, Q2, median), or
    * 75% (second quartile, Q3)
    
      of the values are lower than them. They are less sensitive than the mean to outlier values.

[back to toc](#toc)

<br>

It is all nice to be able to see these numbers, but often data is best explored visually, which is why we will be talking about ...

<br>
<br>

## 3.2 Representing one column - histograms and density line <a id='descr.2'></a>

There exists a number of plotting libraries in python. We will mostly be using **matplotlib** and **seaborn**.

**matplotlib** can be seen as a base library, which defines a lot of low-level plotting functions. **seaborn** is built on top of matplotlib, and provides more high-level functions, which interface very well with DataFrames.

We will not delve in all the details of these two libraries, but will instead try to convey elements about their general philosophies and most common functions/arguments.

That should give you a head start to get the most out of their very well done tutorial and galleries:
 * [matplotlib](https://matplotlib.org/) - [tutorial](https://matplotlib.org/stable/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py) - [gallery](https://matplotlib.org/stable/gallery/index.html)
 * [seaborn](https://seaborn.pydata.org/) - [tutorial](https://seaborn.pydata.org/tutorial.html) - [gallery](https://seaborn.pydata.org/examples/index.html)


> `pandas` also proposes plotting functions, which are basically also using `matplotlib`. 


The simplest way to represent the distribution of a set of values using **seaborn** is to use its **`sns.displot()`** function:


In [ ]:

# "displot" is shorthand for distribution plot
sns.displot( df.Fare )

Above, you should now see a histogram (in blue):
* The *x-axis* corresponds to the fare paid by each passenger.
* The *y-axis* corresponds to the **count**: the number of values falling in a given *bins*
  (i.e. a bar of the histogram).

By using the `kind` argument, one can change the type of plot from histogram `"hist"`, to density line `"kde"`, or cumulative distribution function `"ecdf"`.

For instance :

In [ ]:
sns.displot( df.Fare, kind='kde' )
# Note: kde stands for 'kernel density estimation', which is the method used to compute this density line.

> Note that the y-axis is now in **density** units.

Naturally, many options can be set in order to combine plot types and pimp your plot :

In [ ]:
g = sns.displot(df.Age, kde=True, color='red')

> *Note:* having both the histogram and the density line switches the y-axis from density to count,
  which is sometimes more interpretable.


`sns.displot()` is what we could call a **figure-level** function:
 * it has several *kind* of representation.
 * it does not play nice in **multi-panel** figures.


[back to toc](#toc)

<br>

### Interlude: multi-panel figures <a id='multi'></a>

We create Multi-panel figures using `matplotlib` command and concepts. 
To make it simple, we call `fig, axes = plt.subplots( nb_rows , nb_cols )`, which creates:
 * `fig`, a `figure` object which controls the whole multipanel figure
 * `axes` : a list containing individual `axe` objects though which we can plot on individual panels of the figure


In [ ]:

# Create a list of 100 evenly spaced points between 0 and 10.
x = np.linspace(0, 10, 100) 

# Create a figure with multiple panels: 1 row, 2 columns
fig, axes = plt.subplots( 1, 2 , figsize=(10,8))

print( axes ) # axes is a list of two axes objects

## plotting on the first axe == left panel
axes[0].plot(x, np.sin(x), label='sin')
axes[0].set_title("sine plot")
axes[0].set_xlabel("value")
axes[0].set_ylabel("sine/cosine value")
axes[0].legend()

## plotting on the second axe == right panel
axes[1].plot(x, np.cos(x), label='cos')
axes[1].set_title("cosine plot")
axes[1].set_xlabel("value")
axes[1].set_ylabel("sine/cosine value")

plt.tight_layout() ## this creates a more harmonious organization of the different panels
plt.show()

<br>

### back to data representation ...

The different `kind` (histogram, kde, ecdf) of `sns.displot()` can all be called individually using their own functions : `histplot()`, `kdeplot()`, `ecdfplot()`. 

In contrast to `sns.displot()` which is **figure-level**, these more-specialized functions are *axe-level* : they play nicely in a multiple panel context.

They interface with `matplotlib`'s axes using their `ax` argument.


In [ ]:
# Creating multiple panels : 1 row, 2 columns
fig, axes = plt.subplots( 1, 2 )

# Plotting on the first axe == left panel
sns.histplot(df.Age , ax = axes[0])

# Plotting on the second axe == right panel
sns.kdeplot(df.Fare , ax = axes[1])

plt.tight_layout()   # this creates a more harmonious organization of the different panels
plt.show()

<br>

In **histogram**, a different number of *bins* shows different aspects of the distribution of your data and so it is important to choose one that is appropriate to your sample size and data. 

By default, seaborn automatically infers the number of *bins*. 
You may stick with the original or change it:

In [ ]:
## let's play with a different dataset for this 
dfFractions = pd.read_table("data/census1880_fractions.csv", sep=",")

# Set up a figure with multiple panels, here a 2 by 2 grid
f, axes = plt.subplots( 2, 2, figsize=(14, 7) )

# axes is now a list of list of axes, to represent the 2d nature of our panels:
#  [ [<axe>,<axe>] ,
#    [<ax>,<axe>]]


# then you can specify where each plot goes on the figure with the ax argument of the ([0,0] is the top left corner)

# Plot a simple histogram with binsize determined automatically
sns.histplot(dfFractions['0-14 y.o.'], kde=False, color="b",
             ax=axes[0, 0]).set_title('automatic')

# Plot a simple histogram with binsize 5, 10 , 1000
sns.histplot(dfFractions['0-14 y.o.'], bins=5   , kde=False, color="b", ax=axes[0, 1]).set_title('5 bins')
sns.histplot(dfFractions['0-14 y.o.'], bins=10  , kde=False, color="b", ax=axes[1, 0]).set_title('10 bins')
sns.histplot(dfFractions['0-14 y.o.'], bins=1000 , kde=False, color="b", ax=axes[1, 1]).set_title('1000 bins')

plt.tight_layout()# this makes the panels margins and setup more graceful


So, to recap, we use `sns.histplot` instead of `sns.displot` : 
 * `histplot` : gives a lower-level kind of plot which is easier to manipulate in multiple figure.
 * `displot` : high-level function, with a lot of capabilities, but does not play nice if not the top figure.
     


<br>

We can combine graphical elements coming from `matplotlib` and `seaborn` quite seamlessly.

Here is a more advanced example where I represent the the **mean**, the **median** and the **mode** of a distribution on top of an histogram.


In [ ]:


# Here we just define a small function for plotting a distribution with the mean median and mode 
def plotWithMeanMedianMode( dat , ax):
    """ 
        Takes:
            * dat : a pandas series
            * ax : a matplotlib axe    
    """
    mode=dat.mode()[0] #we only select the first mode
    mean=dat.mean()
    median=dat.median()

    sns.histplot( dat , kde=True , ax=ax) # line for histogram and density line

    ax.axvline(mean, color='r', linestyle='--' , label='Mean')
    ax.axvline(median, color='g', linestyle='-' , label='Median')
    ax.axvline(mode, color='b', linestyle='-' , label='Mode')
    ax.legend()


In [ ]:
f, axes = plt.subplots( 2, 1, figsize=(15, 10) )

plotWithMeanMedianMode( df.Fare , ax=axes[0])
plotWithMeanMedianMode( df.Age , ax=axes[1])
plt.show()

Of note here :
 * `axvline`: create a vertical line.
 * `label`: legend label associated with the element created by this function.
   Common to a lot of matplotlib plotting functions.
 * `ax.legend()`: makes the legend appear.
 
> You can specify where the legend goes with `loc=` one of `{'best', 'upper left', 'upper right', 'lower left', 'lower right', 'upper center', 'lower center', 'center left', 'center right', 'center'}` or a `(x,y)` tuple  (more options to manually specify elements in the `help()`)

[back to toc](#toc)

<br>
<br>

## Exercise 2 - histograms

Using the Titanic dataset:

1. Plot the `Age` distribution among first class passengers. Try to choose an appropriate mode of 
   representation (histogram? density line? number of bins?).
2. Make a figure with 3 panels. In the panels, plot the histogram of the `Fare` among passengers in
   the first, second, and third class, respectively.


Solutions:

In [ ]:
# %load -r 1-6 solutions/solution_01_02.py
# 1. Plot the Age distribution among first class passengers. Try to choose an appropriate mode of representation (histogram? density line? number of bins?)


In [ ]:
# %load -r 7- solutions/solution_01_02.py
#2. Make a figure with 3 panels. In the panels, plot the histogram of the `Fare` among passengers in
#   the first, second, and third class, respectively.

<br>
<br>

[back to toc](#toc)

## 3.3 Accounting for categories in the data <a id='descr.3'></a>

Here the categories would be the passenger class of sex, but depending on data-sets they could be a genotype (Wild-Type versus KO), sex, experimental condition, ...

Of course, we could manually compute the means and plot an histogram for each category in our data-set, but pandas and seaborn offer much more efficient routines for this.


Computing a summary statistics on a pandas DataFrame is done **using the `df.groupby(...)` method and the applying some function** to the result of that grouping.

In [ ]:
grouped = df.groupby('Sex')     # grouping by Sex.
grouped['Age'].median()         # median age.

So we get the median age per Sex. 

<br>

Of course here there are only two categories, but this gets very interesting when there is a lot of categories.

Remember our swiss census from 1880?


In [ ]:
df_census = pd.read_table("data/swiss_census_1880.csv", sep=",")

grouped = df_census.groupby("canton name")
grouped["Catholic"].sum()                     # number of catholics per canton in 1880.

### Micro-Exercise:
* Apply the next cell, which makes a copy of the DataFrame and adds a new column named "Age_category"
* Compute survival rates by Sex, Age_category, and Pclass.

In [ ]:
dfc = df.copy()
def age_category(x):
    age_classes = {"child": 12, "teenager": 17, "adult": 64, "senior": 200}
    for label, threshold in age_classes.items():
        if x <= threshold:
            return label
dfc['Age_category'] = dfc.Age.apply( age_category )


In [ ]:
dfc.head()

<br>

Computing these summary statistics is all good, but you can also vizualize them using `seaborn` argument `hue` : 



In [ ]:
sns.displot( x='Age', hue='Sex', data=df, kind='kde')

`hue` is quite ubiquitous in seaborn functions, and lets you determine a categorical column by which you want to split your representation.

Note how we switched our syntax too: we now use column names, with a `data=` argument specifying from which `DataFrame` the data comes.

<br>

The default seaborn scheme is fairly nice, but maybe you want to change it :


In [ ]:
f, axes = plt.subplots( 2, 2, figsize=(14, 7) )

# default
sns.kdeplot( x='Age' , hue = 'Sex' , linewidth=3, data=df , ax = axes[0][0])

# using an existing seaborn palette
sns.set_palette(sns.color_palette('Set2'))
sns.kdeplot( x='Age' , hue = 'Sex' , data=df , linewidth=3, ax= axes[0][1])


# setting manually, with hex values
sns.kdeplot( x='Age' , hue = 'Sex' , data=df , linewidth=3, ax= axes[1][0] , 
           palette = ['#FFCC04','#1F8AB3'])


# setting semi manually, using the xkcd palette https://xkcd.com/color/rgb/
sns.kdeplot( x='Age' , hue = 'Sex' , data=df , linewidth=3, ax= axes[1][1] , 
           palette = [ 'xkcd:lavender' , 'xkcd:dark mint' ])


> *Note:* use `linewidth=` to get wider/narrower lines

You can learn more on all the options there :
 * [matplotlib colors](https://matplotlib.org/stable/tutorials/colors/colors.html)
 * [seaborn colors](https://seaborn.pydata.org/tutorial/color_palettes.html)
 

<br>

Sometimes, the `displot` options don't really cut it, perhaps because you have many diffrent categories, or maybe because you want to have more than one column to do the categorization.

This is when **`sns.catplot()`**, another **figure-level** function with a lot a `kind`, saves the day!


In [ ]:
sns.catplot( x='Fare' , y = 'Pclass' ,
            orient='horizontal' , 
            data=df , aspect = 2.0 , height = 7)

Like `sns.displot`, `sns.catplot` is a master function which is designed to create a full figure at once rather than be embedded in a larger figure. 

This is why its arguments are a bit different when it comes to setting the figure *height and width*, which is done using :
 * `height` : height of the figure (no trick there)
 * `aspect` : width/height ratio of the figure ( high aspect -> wider figure )

The `kind` argument let's you control the overall look of the plot. I
It can be:
* 'strip' : this is the default, showing all data points. 
* **'box'** : the famous boxplot
* **'violin'** : an alternative to the boxplot using density lines
* **'bar'** : the (in)famous barplot
* 'swarm' : similar to 'strip' but with another way of aranging the points
* 'boxen' : some intermediary between a boxplot and a violin plot
* 'point' : alternative to the barplot where only the top point is shown



In [ ]:
kinds = ['box','violin','bar','boxen','strip','point']  #,'swarm'] # swarm takes a long time to compute

for i,k in enumerate(kinds):
    sns.catplot( x='Fare', y='Pclass', orient='horizontal', data=df, kind=k, aspect=5 , height=2)


So here we can see:
 * **boxplot:** represent quartiles and a few other things
 * **violinplot:** density line plot with an (optional) boxplot in the center 
 * **barplot:** just shows the mean + an error bar (by default 95% CI computed using bootstrapping)
 * **boxen:** the unholy child of a boxplot and a violinplot
 * **strip:** the default, shows the points (super nice, except when you have a lot of points)
 * **point:** basically a barplot without bars and a line between categories

Given all this, one can reasonnably ask the question : **which one is the *best* ?**

While the answer is not clear about the best way to represent a distribution over one or multiple categories, we definitely have some [good answers regarding bad/misleading ways](https://stekhoven.shinyapps.io/barplotNonsense/) (hint, barplot does not shine).  




<br>

Anyhow the `hue` argument can also be used here to great effect:

In [ ]:
sns.catplot( x='Fare' , 
            y = 'Pclass' , 
            hue='Sex' , 
            orient='horizontal' , 
            kind='box', data=df , aspect = 2.0 , height = 4)
## it is also very nice with kind='point'

<br>

## Exercice 3 - representing categories

Using the 1880 swiss census data:

1. compute a new column "fraction60+" representing the fraction of 60+ years old people in each town.
    **hint :** column `'60+ y.o.'` contains the number of 60+ years old ; column `'Total'` contains
    the total number of inhabitant
    
2. Represent the proportion of people more 60 years old (`'60+ y.o.'`) across all cantons.
   Choose the most appropriate kind of plot.

In [ ]:
df_census = pd.read_table("data/swiss_census_1880.csv", sep=",")
df_census.head()

In [ ]:
# %load -r 1-6 solutions/solution_01_03.py
# 1.


In [ ]:
# %load -r 7- solutions/solution_01_03.py
# 2.

Of course possibilities are endless. Here is a fancy one, inspired by [this](https://seaborn.pydata.org/examples/kde_ridgeplot.html) and its [later correction](https://www.pythonfixing.com/2022/02/fixed-python-seaborn-ridge-plot.html).

In [ ]:
# %load solutions/solution_01_03_fancy.py
# fancy solution inspired by  https://seaborn.pydata.org/examples/kde_ridgeplot.html

<br>
<br>

[back to toc](#toc)

## 3.4 Representing the relationship between 2 numerical variables <a id="descr.4" ></a>

**`sns.scatterplot()`** lets us represent the relationship between two numerical variables.

In [ ]:
df = pd.read_table("data/titanic.csv", sep=",") ## just in case we need to re-read  the data

In [ ]:
#reset the seaborn default theme
sns.reset_orig()

In [ ]:
fig = plt.figure(figsize=(14,6))
ax = sns.scatterplot(x='Fare', y='Age', data=df)

OK, it is not so nice... let's set the x-axis to a log scale :

In [ ]:
fig = plt.figure(figsize=(14,6))
ax = sns.scatterplot( x = 'Fare' , y = 'Age' , data=df )
ax.set(xscale="log")      # setting axis to log scale.
ax.set_xlim( (3,1000) )   # manually setting the limit of the x axis.

Of course, we can customize :

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = sns.scatterplot( x = 'Fare' , y = 'Age' , 
                     hue='Pclass' , 
                     palette=['xkcd:tomato',
                              'xkcd:teal','xkcd:mustard'],
                     style='Sex', data=df , s=100 )
ax.set(xscale="log")      # setting axis to log scale.
ax.set_xlim( (3,1000) )   # manually setting the limit of the x axis.


> *Note:* `style=` associates the marker shape to a column.

Finally, a little data exploration trick : **`sns.pairplot()`**

In [ ]:
sns.pairplot(df_census.iloc[:,3:7] )

<br>
<br>
<br>

[back to toc](#toc)


# 4. Writing data and plots to disk <a id='writing'></a>
-------------------------------------------------

To write your data-set to a file, use:

In [ ]:
df.to_csv('myData.csv')

As you can see, it is about as simple as reading. Actually, it is really similar.
You have several functions :
* `to_csv()`: write DataFrame as comma-separated file, or any other separator-delimited format such as
  tab-delimited. 
* `to_excel()`: write DataFrame in Excel format.
* `to_html()`: write DataFrame in HTML format.
* More [writer functions are available](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)

<br>

... and similar arguments : when using the `to_csv()` writer functions, some useful arguments are:
* `sep`: the type of delimiter to use. By default, `sep=","`. To write a tab-delimited file e.g., one would
  set `sep="\t"`.
* `header=None` to not include the header in the exported file.


<br>

To save figures to disk, we rely on the **`savefig()`** method:
* Different output formats can be specified by changing the output file name extension, 
  e.g. `.pdf`, `.svg`, `.jpg`, ...
* For raster formats, the `dpi` argument can help define the image quality.
* See `help(plt.savefig)` or [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html?highlight=savefig#matplotlib.pyplot.savefig) for more info.

In [ ]:
k='box'

# Make you plot as usual, redirecting it to a variable (my_plot)
represented_variable , category = 'Total' , 'majority language'
my_plot = sns.catplot( x = represented_variable , y= category ,
             data=dfFractions , kind = k , orient='h',height=10, aspect=2 )

# Save plot to disk, using the savefig() method:
my_plot.savefig("output.png")

In [ ]:
help(plt.savefig)

In [ ]:
# For multipanel figures:
f, axes = plt.subplots(2, 2, figsize=(7, 7))

sns.histplot(dfFractions['0-14 y.o.'], kde=False, color="b", ax=axes[0, 0]).set_title('automatic')
sns.histplot(dfFractions['0-14 y.o.'], bins=5   , kde=False, color="b", ax=axes[0, 1]).set_title('5 bins')
sns.histplot(dfFractions['0-14 y.o.'], bins=10  , kde=False, color="b", ax=axes[1, 0]).set_title('10 bins')
sns.histplot(dfFractions['0-14 y.o.'], bins=1000 , kde=False, color="b", ax=axes[1, 1]).set_title('1000 bins')
plt.tight_layout()# this makes the panels margins and setup more graceful

f.savefig('output_multipanel.pdf')

<br>
<br>
<br>

[back to toc](#toc)

# 5. Free form exercise <a id='exo'></a>
---------------------------------

The goal of this exercise is to perform an exploration of some data related to heart disease.

In particular, we want to explore the relationship between a `target` variable - whether patient has a heart disease or not - and several other variables such as cholesterol level, age, ...

The data is present in the file `'data/heartData_simplified.csv'`, which is a cleaned and simplified version of the [UCI heart disease data set](https://archive.ics.uci.edu/ml/datasets/heart+Disease)


### Description of the columns

* age : Patient age in years
* sex : Patient sex
* chol : Cholesterol level in mg/dl. 
* thalach : Maxium heart rate during the stress test
* oldpeak : Decrease of the ST segment during exercise according to the same one on rest.
* ca : Number of main blood vessels coloured by the radioactive dye. The number varies between 0 to 3.
* thal : Results of the blood flow observed via the radioactive dye.
	* defect -> fixed defect (no blood flow in some part of the heart)
	* normal -> normal blood flow
	* reversible -> reversible defect (a blood flow is observed but it is not normal)
* target : Whether the patient has a heart disease or not

### Instructions

As stated earlier, your goal is to explore this data-set. 
One objective of this would be to diagnose eventual problems in this dataset (outliers, strange values) and prepare further statistical analysis and reporting.

To this end you will want to formulate a number of hypothesis that would be interesting to pursue from this data (*e.g.*, is heart disease linked to  cholesterol levels), and gather evidence (plots, summary statistics) explaining why this hypothesis seems to be worth testing for.

> Note : we do not ask you to perform the statistical testing itself. Do it if you feel like it.

We will not provide a particular set of precise questions, but here are a few checkpoints to help you get stared :

* Read the data as a pandas `DataFrame`
* Compute summary statistics for the different variables 
* eventually, do the same for different subset of the data (for instance, grouping by sex)
* use visualization to help you describe the relationship between the different variables
* choose a few associations (2?4?) that seems promising and describe them


<br>
<br>
<br>

[back to toc](#toc)


# 6. tips and tricks <a id='tricks'></a>
---------------------------

This is a collection of nice tricks that don't really blong anywhere but are nice to know

## 6.1 Applying custom functions to a DataFrame by rows or columns

As we have just seen, pandas DataFrame have a number of built-in methods - e.g. `describe()`, `count()`, `mean()` - that can be applied row-wise or column-wise.

But it is also possible to apply any custom function on rows/columns using `apply(func, axis)`:
* `func`: the function to apply.
* `axis=0` to apply the function row-wise (this is the default), or `axis=1` to apply it column-wise.

For example, let's implement our own version of the built-in [`mean()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html).

**Note:** the `apply()` function only works on DataFrames. To apply a custom function to each element in a Series 
(e.g. a single column of a DataFrame), the `map()` function can be used. E.g.: `df["Survived"].map(func)`

<br>

**Example**

In [ ]:
# Print the DataFrame as a reminder:
df.head()

In [ ]:
def longest_str(seq):
    max_len = -1
    max_value = ''
    for x in seq:
        length = len(str(x))
        if length > max_len:
            max_len = length
            max_value = x
    return max_value

# Test of the longest_str() function on dummy data:
print("The longest string is:", longest_str(("eggs", "hello world", "helloworld", "spam")), "\n")

In [ ]:
# Apply custom function by column (i.e row with the most characters in each column):
print(df.apply(longest_str, axis=0), "\n")

In [ ]:
# Apply custom function by row (i.e column with the most characters in each row):
print(df.apply(longest_str, axis=1), "\n")

<br>

## 6.2 Sorting operations on dataframes

DataFrames can be sorted on one or more specific column(s) using **`sort_values()`**:

In [ ]:
df_sc = df_sc.sort_values(df_sc.columns[0], ascending=False)
df_sc.head()

<br>

Or we can sort by index using **`sort_index()`**:

In [ ]:
df_sc.sort_index(ascending=True).head()

Finding the maximum value in each column. Note the `axis` parameter. This gives the dimension along which values are compared. `axis=0` indicates that the comparison is across rows and there looping over all index values in all other dimensions. In this case, for each column we got a value.

In [ ]:
df_sc.max(axis=0)

Let's now find the max in each row (in this case for each gene), over all columns (samples):

In [ ]:
df_sc.max(axis=1).head()

Maybe what we want is not the maximum value but the index at which it is found: 

In [ ]:
df_sc.idxmax(axis=1).head()

<br>
<br>

## 6.4 Merge and join DataFrames

The **[`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)** and **[`join()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html)** methods allow to combine DataFrames, linking their rows based on a common column (other referred to as a **key**).

To illustrate these 2 methods, let's create 2 DataFrames that we can merge.
> *Note:* this also illustrates how **a dataframe can be constructed from a dictionary** data structure.
  The dictionary keys are treated as column names, and the list of values associated with a key is
  treated as list of elements in the corresponding column. Note that all columns should have the same
  number of elements (or a single element, in which case all rows of the column contain this same
  element).  
  If no index is specified, pandas uses its default indexing, i.e. row positions.

In [ ]:
df1 = pd.DataFrame({
    'key': ['b','b','a','c','a','a','b'], 
    'data1': range(7)
})
df1

In [ ]:
df2 = pd.DataFrame({
    'key': ['a','b','d'], 
    'data2': range(3)
})
df2

Let's merge the two data frames, with the default application of the `merge()` function:

In [ ]:
pd.merge(df1, df2)

How has python interpreted our call to `merge()`?

1. It has assumed that we want to merge on the basis of the common `key` column.
2. It has identified the values of `key` which occur in both dataframes.
3. It has generated a dataframe with all combinations of rows from dataframes 1 and 2 that are 
   associated with a particular `key` value.

We can be more precise by specifying how to merge the dataframes, using the **`on`** option:

In [ ]:
pd.merge(df1, df2, on="key")

By default, merge performs an "inner" operation, taking the intersection of the key sets. However, we can specify the way we want to merge by passing `"outer"`, `"left"`, `"right"` to the **`how`** argument. This determines which set of keys to consider (the union of the two sets, all of those that occur in df1, all of those that occur in df2). Missing values show up as `NaN`.

In [ ]:
pd.merge(df1, df2, on="key", how="outer")

Merging can also be done based on the index values. Let's illustrate this using the mouse heart gene expression dataset we used earlier:

In [ ]:
df = pd.read_table("data/heartData_simplified.csv")
df.head()

Let's create two data frames, one containing the data for the WT and the other for the KO:

In [ ]:
wt_cols = [col for col in df.columns if 'WT' in col and 'avg' not in col and 'log' not in col]
ko_cols = [col for col in df.columns if 'KO' in col and 'avg' not in col and 'log' not in col]
df_WT = df[wt_cols]
df_KO = df[ko_cols]
print(df_WT.head())
print(df_KO.head())

Let's merge these frames based on the index:

In [ ]:
merged_df = pd.merge(df_WT, df_KO, left_index=True, right_index=True)
merged_df.head()

<br>

## 6.5 wide and long format

**Wide :**

| Id     | mol1   | mol2    |
| ------ |:------:| -------:|
| a      | 1.0    | 10.0    |
| b      | 2.0    |   20.0  |
| c      | 3.0    |    30.0 |

**Long :**

| Id     | Value  | Attr    |
| ------ |:------:| -------:|
| a      | 1.0    | mol1    |
| a      | 10.0   | mol2    |
| b      | 2.0    | mol1    |
| b      | 20.0   | mol2    |
| c      | 3.0    | mol1    |
| c      | 30.0   | mol2    |


Sometimes it can be interesting to go from wide to long or long to wide, because some operations are easier on one format or the other.

In [ ]:
# Data in wide format:
df_sc.head()

In [ ]:
df_sc["gene"] = df_sc.index
df_sc_long = pd.melt(df_sc, id_vars=['gene']) # to long format, with gene as identifiers 
df_sc_long.head()

In [ ]:
# This now allows us to do this, which was a bit difficult otherwise :
df_sc_long['logVal'] = np.log10(10**0 + df_sc_long['value'])

g = sns.catplot( x='logVal' , y='variable' , orient='horizontal' , data=df_sc_long , 
                kind='bar',
                 aspect = 2 , height = 7 )

## 6.6 seaborn themes + python graph gallery

Seaborn and matplotlib offer nigh-endless possibilities when it comes to costumizing graph elements.

A simple way to change the general feeling of a figure is to play with [seaborn themes](https://seaborn.pydata.org/tutorial/aesthetics.html), or [matplotlib styles](https://www.dunderdata.com/blog/view-all-available-matplotlib-styles).

In general, when composing a figure we often do not start from nothing, and work our ways from existing examples.
Here are useful resources to do just that:
- [matplotlib gallery](https://matplotlib.org/stable/gallery/index.html)
- [seaborn gallery](https://seaborn.pydata.org/examples/index.html)
- [python-graph-gallery](https://www.python-graph-gallery.com/)